In [2]:
from pyspark.sql.functions import *
from  pyspark.sql.functions import abs
import time
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
df = spark.read.csv("hdfs://orion11:11101/3hr_sample/sampled_2015/", sep=r'\t', header=True, inferSchema="true")

In [4]:
df.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface
0,1422716400000,27.764760,-73.670418,6.00,null,16526.799,102252.0,17187.525,6.0,0.0000,296.64557,208.29962,null,0.0,0.0,24227.295,0.00000,null
1,1422716400000,39.376466,-125.097760,6.00,null,22126.799,101660.0,19787.525,2.0,0.0000,286.77057,210.79962,null,0.0,0.0,24227.295,0.00000,null
2,1422716400000,23.580932,-120.938083,6.00,null,23726.799,101649.0,10587.525,1.0,0.0000,294.27057,200.67462,null,0.0,0.0,24227.295,0.00000,null
3,1422716400000,59.622597,-123.937086,30.25,null,23726.799,99260.0,32587.525,99.0,0.3966,243.14555,222.67462,null,0.0,11.5,24227.295,0.06625,null
4,1422716400000,33.498223,-74.429542,6.00,null,24926.799,102410.0,22587.525,94.0,0.0000,294.64557,215.92462,null,0.0,0.0,24227.295,0.00000,null


In [5]:
from math import log10

#  Note: the alphabet in geohash differs from the common base32 alphabet described in IETF's RFC 4648
# (http://tools.ietf.org/html/rfc4648)

__base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
__decodemap = dict()
for i in range(len(__base32)):
    __decodemap[__base32[i]] = i


def decode_exactly(geohash):
    """
    Decode the geohash to its exact values, including the error
    margins of the result.  Returns four float values: latitude,
    longitude, the plus/minus error for latitude (as a positive
    number) and the plus/minus error for longitude (as a positive
    number).
    """
    lat_interval, lon_interval = (-90.0, 90.0), (-180.0, 180.0)
    lat_err, lon_err = 90.0, 180.0
    is_even = True
    for c in geohash:
        cd = __decodemap[c]
        for mask in [16, 8, 4, 2, 1]:
            if is_even: # adds longitude info
                lon_err /= 2
                if cd & mask:
                    lon_interval = ((lon_interval[0]+lon_interval[1])/2, lon_interval[1])
                else:
                    lon_interval = (lon_interval[0], (lon_interval[0]+lon_interval[1])/2)
            else:      # adds latitude info
                lat_err /= 2
                if cd & mask:
                    lat_interval = ((lat_interval[0]+lat_interval[1])/2, lat_interval[1])
                else:
                    lat_interval = (lat_interval[0], (lat_interval[0]+lat_interval[1])/2)
            is_even = not is_even
    lat = (lat_interval[0] + lat_interval[1]) / 2
    lon = (lon_interval[0] + lon_interval[1]) / 2
    return lat, lon, lat_err, lon_err


def decode(geohash):
    """
    Decode geohash, returning two float with latitude and longitude
    containing only relevant digits and with trailing zeroes removed.
    """
    lat, lon, lat_err, lon_err = decode_exactly(geohash)
    # Format to the number of decimals that are known
    lats = "%.*f" % (max(1, int(round(-log10(lat_err)))) - 1, lat)
    lons = "%.*f" % (max(1, int(round(-log10(lon_err)))) - 1, lon)
    if '.' in lats: lats = lats.rstrip('0')
    if '.' in lons: lons = lons.rstrip('0')
    return float(lats), float(lons)


def encode(latitude, longitude, precision=12):
    """
    Encode a position given in float arguments latitude, longitude to
    a geohash which will have the character count precision.
    """
    lat_interval = (-90.0, 90.0)
    lon_interval = (-180.0, 180.0)
    geohash = []
    bits = [16, 8, 4, 2, 1]
    bit = 0
    ch = 0
    even = True
    while len(geohash) < precision:
        if even:
            mid = (lon_interval[0] + lon_interval[1]) / 2
            if longitude > mid:
                ch |= bits[bit]
                lon_interval = (mid, lon_interval[1])
            else:
                lon_interval = (lon_interval[0], mid)
        else:
            mid = (lat_interval[0] + lat_interval[1]) / 2
            if latitude > mid:
                ch |= bits[bit]
                lat_interval = (mid, lat_interval[1])
            else:
                lat_interval = (lat_interval[0], mid)
        even = not even
        if bit < 4:
            bit += 1
        else:
            geohash += __base32[ch]
            bit = 0
            ch = 0
    return ''.join(geohash)

In [6]:
def convertToGeoHash(lat, lon):
    return encode(lat, lon,3)

udfConvertToGeoHash = F.udf(convertToGeoHash, StringType())

geohashDf = df.withColumn("5_hash", udfConvertToGeoHash("2_lat", "3_lon"))

geohashDf.limit(5).toPandas()

#geohashDf = comfortPressure.withColumn("2_lat", encode("2_lat", "3_lon")))
#geohashDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,1422716400000,27.764760,-73.670418,6.00,null,16526.799,102252.0,17187.525,6.0,0.0000,296.64557,208.29962,null,0.0,0.0,24227.295,0.00000,null,dkg
1,1422716400000,39.376466,-125.097760,6.00,null,22126.799,101660.0,19787.525,2.0,0.0000,286.77057,210.79962,null,0.0,0.0,24227.295,0.00000,null,9pp
2,1422716400000,23.580932,-120.938083,6.00,null,23726.799,101649.0,10587.525,1.0,0.0000,294.27057,200.67462,null,0.0,0.0,24227.295,0.00000,null,9k1
3,1422716400000,59.622597,-123.937086,30.25,null,23726.799,99260.0,32587.525,99.0,0.3966,243.14555,222.67462,null,0.0,11.5,24227.295,0.06625,null,c4x
4,1422716400000,33.498223,-74.429542,6.00,null,24926.799,102410.0,22587.525,94.0,0.0000,294.64557,215.92462,null,0.0,0.0,24227.295,0.00000,null,dmg


In [7]:
def convertToMonth(value):
    return time.strftime('%m', time.localtime(int(value)/1000))

udfConvertToMonth = F.udf(convertToMonth, StringType())

monthDf = geohashDf.withColumn("1_time", udfConvertToMonth("1_time"))
monthDf.limit(5).toPandas()

,1_time,2_lat,3_lon,albedo_surface,precipitable_water_entire_atmosphere_single_layer,pressure_maximum_wind,pressure_surface,pressure_tropopause,relative_humidity_zerodegc_isotherm,snow_depth_surface,temperature_surface,temperature_tropopause,total_cloud_cover_entire_atmosphere_single_layer,total_precipitation_surface_3_hour_accumulation,vegetation_surface,visibility_surface,wilting_point_surface,wind_speed_gust_surface,5_hash
0,01,27.764760,-73.670418,6.00,null,16526.799,102252.0,17187.525,6.0,0.0000,296.64557,208.29962,null,0.0,0.0,24227.295,0.00000,null,dkg
1,01,39.376466,-125.097760,6.00,null,22126.799,101660.0,19787.525,2.0,0.0000,286.77057,210.79962,null,0.0,0.0,24227.295,0.00000,null,9pp
2,01,23.580932,-120.938083,6.00,null,23726.799,101649.0,10587.525,1.0,0.0000,294.27057,200.67462,null,0.0,0.0,24227.295,0.00000,null,9k1
3,01,59.622597,-123.937086,30.25,null,23726.799,99260.0,32587.525,99.0,0.3966,243.14555,222.67462,null,0.0,11.5,24227.295,0.06625,null,c4x
4,01,33.498223,-74.429542,6.00,null,24926.799,102410.0,22587.525,94.0,0.0000,294.64557,215.92462,null,0.0,0.0,24227.295,0.00000,null,dmg


In [8]:
groupedDf = monthDf.groupBy("5_hash","1_time").agg(F.avg("temperature_surface").alias("temperature_surface"),F.avg("relative_humidity_zerodegc_isotherm").alias("relative_humidity_zerodegc_isotherm"),F.avg("pressure_surface").alias("pressure_surface"))

In [9]:
groupedDf.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,9x8,02,273.681613,74.255650,81600.053672
1,drt,02,266.201950,74.872283,100363.101902
2,c1x,02,267.402003,89.372813,90690.963661
3,9s4,02,296.416733,20.135613,101299.205189
4,9gk,02,295.691544,24.731602,101289.878788


In [10]:
comfortTemp = groupedDf.filter(groupedDf.temperature_surface > 293).filter(groupedDf.temperature_surface < 304)
comfortTemp.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,dq4,07,301.827644,47.334716,101476.288507
1,9vu,07,303.129802,31.384220,100030.138937
2,drh,07,295.977855,38.573972,101354.578883
3,f12,07,293.582193,58.016265,97626.793976
4,dhv,07,303.171282,61.260818,101530.117961


In [11]:
comfortHumid = comfortTemp.filter(comfortTemp.relative_humidity_zerodegc_isotherm < 25).filter(comfortTemp.relative_humidity_zerodegc_isotherm > 5)
comfortHumid.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,dk4,04,300.093168,22.494709,101529.432099
1,8vj,04,293.166980,22.879397,102454.649246
2,d7f,04,300.563157,17.690395,101473.437393
3,d5p,04,300.842084,12.500845,101302.632601
4,9hh,10,298.623732,12.896455,101519.120428


In [12]:
comfortPressure = comfortHumid.filter(comfortHumid.pressure_surface > 101000).filter(comfortHumid.pressure_surface < 102000)
comfortPressure.limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface
0,d7y,01,299.870088,19.743363,101834.309214
1,9h0,01,295.412314,22.947572,101769.731788
2,d6c,01,300.682380,14.061905,101536.663095
3,dk4,04,300.093168,22.494709,101529.432099
4,d7f,04,300.563157,17.690395,101473.437393


In [13]:
#temperature_surface (298 - a)
#relative_humidity_zerodegc_isotherm (15 - b)
#pressure_surface(101500 - c)
# from  pyspark.sql.functions import *
from  pyspark.sql.functions import abs

def getComfortIndex(temperature_surface, relative_humidity_zerodegc_isotherm, pressure_surface):
    if temperature_surface < 298:
        x = 298 - temperature_surface
    else:
        x = temperature_surface -298
        
    if relative_humidity_zerodegc_isotherm < 15:
        y = 15 - relative_humidity_zerodegc_isotherm
    else:
        y = relative_humidity_zerodegc_isotherm -15
        
    if pressure_surface < 101500:
        z = 101500 - pressure_surface
    else:
        z = pressure_surface -101500
    
    return (x + y + z)/3

udfConvertToComfortIndex = F.udf(getComfortIndex, StringType())

comfortIndexDf = comfortPressure.withColumn("c_idx", udfConvertToComfortIndex("temperature_surface", "relative_humidity_zerodegc_isotherm","pressure_surface"))

In [14]:
comfortIndexDf.orderBy("c_idx").limit(5).toPandas()

,5_hash,1_time,temperature_surface,relative_humidity_zerodegc_isotherm,pressure_surface,c_idx
0,d7b,01,298.348927,16.909580,101501.593649,1.2840521994996514
1,9vp,10,298.301239,23.413302,101478.392518,10.107340817496015
2,95x,11,298.109089,19.937802,101474.337802,10.23636284003633
3,d6v,03,300.360270,5.042528,101518.578263,10.298668363849922
4,9s0,02,296.744381,18.621229,101526.536313,10.471053765362091


In [16]:
comfortIndexDf.limit(5).repartition(1).write.option("delimiter", "\t").csv("hdfs://orion11:11101/output/02")

print("done")

done
